# 算法基础

## 线性回归

### 正规方程法

$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n
$

简化模式  
$
\hat{y} = h_{\mathbf{\theta}}(\mathbf{x}) = \mathbf{\theta}^T \cdot \mathbf{x}
$

MSE损失函数  
$
\text{MSE}(\mathbf{X}, h_{\mathbf{\theta}}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{(\mathbf{\theta}^T \cdot \mathbf{x}^{(i)} - y^{(i)})^2}
$

该损失函数拥有一个闭式解,其正规方程为:  
$
\hat{\mathbf{\theta}} = (\mathbf{X}^T \cdot \mathbf{X})^{-1} \cdot \mathbf{X}^T \cdot \mathbf{y}
$

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]  #在每一个实例的第一列添加 x0 = 1 
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

np.linalg.inv(): 计算一个矩阵的逆矩阵  
A.dot(B) : A矩阵与B矩阵的点乘

scikit-learn:LinearRegression

当特征数量变得非常庞大时(e.g.,100,000),正规方程的计算时间会变得非常长

### 梯度下降法

线性回归的MSE损失函数是一个凸函数,这表示其中只有全局最小值,而没有局部最小值  
同时,该损失函数还是一个斜率不会突然改变的连续函数  
这两个特征保证了梯度下降法一定能找到全局最小值

### 批量梯度下降(Batch Gradient Descent)

计算损失函数的偏导  
$
\dfrac{\partial}{\partial \theta_j} \text{MSE}(\mathbf{\theta}) = \dfrac{2}{m}\sum\limits_{i=1}^{m}(\mathbf{\theta}^T \cdot \mathbf{x}^{(i)} - y^{(i)})\, x_j^{(i)}
$  
j特征方向上的梯度

当然,在实际使用时,可以一次性计算所有特征的梯度   
$
\nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta}) =
\begin{pmatrix}
 \frac{\partial}{\partial \theta_0} \text{MSE}(\mathbf{\theta}) \\
 \frac{\partial}{\partial \theta_1} \text{MSE}(\mathbf{\theta}) \\
 \vdots \\
 \frac{\partial}{\partial \theta_n} \text{MSE}(\mathbf{\theta})
\end{pmatrix}
 = \dfrac{2}{m} \mathbf{X}^T \cdot (\mathbf{X} \cdot \mathbf{\theta} - \mathbf{y})
$  
其中,梯度向量记为$ \nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta}) $

注:在该公式形式下,在每一步梯度下降时,都计算了一整个训练集$\mathbf{X}$  
因此,当训练集样本数量非常大时,该方法速度非常慢

当得到了梯度向量(指向上升),只要取其相反数便能得到梯度下降值  
$
\mathbf{\theta}^{(\text{next step})} = \mathbf{\theta} - \eta \nabla_{\mathbf{\theta}}\, \text{MSE}(\mathbf{\theta})
$  
其中,$\eta$表示学习率,学习率太小的话需要迭代太多次达到最优解,学习率太大的话则会跳过最优解并逐渐偏离

In [ ]:
eta = 0.1
n_iterations = 1000
m = 100
theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

另一个可调的参数是迭代次数  
迭代次数太少的话,可能无法达到最优解,迭代次数太多的话,则可能达到最优解后仍然在继续迭代    
一个简单的解决方案是,设置一个较大的迭代次数,中途一旦梯度向量小于tolerance(一个比较小的值)时,则停止迭代

### 随机梯度下降法(Stochastic Gradient Descent)

随机梯度下降法相比普通的梯度下降法更为无序  
在训练的过程中,损失函数时而下降,时而上升,但总体上损失函数仍逐渐下降    
随着不断训练,损失函数将逐渐接近最小值,但无法到达最小值,因此最终得到的参数并不是最优解,而是较为接近最优解的解

正因为随机梯度下降法无序的特性,它能够帮助算法跳出局部最小点,从而拥有更大的机会找到全局最小点

#### 模拟退火法(simulated annealing)

为了能够找到尽可能接近最优解的解,一种解决方案是采用逐渐降低的学习率  
一开始采用较大的值,用以加快训练进程并跳出局部最小点  
之后逐渐降低,最终不断趋近于最优解, 该方法称为模拟退火法

决定每次迭代学习率的函数称为learning schedule

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

In [ ]:
#scikit-learn中的随机梯度下降法
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(n_iter=50, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

n_iter=50 表示运行50次epoch  
penalty=None 表示不适用任何正则化  
eta0=0.1 表示初始学习率为0.1

### 小批量梯度下降法(Mini-batch Gradient Descent)

每次计算样本集中随机选出的小批量数据的梯度  
该方法的好处是,相比于随机梯度下降,能够从矩阵操作的硬件优化上得到性能的极大提升

在不规则性上,该方法要小于SGD,因此mini-batch GD的解要更靠近最优解  
然而,这一特性使得mini-batch GD更难逃离局部最小点

## 多项式回归(Polynomial Regression)

In [ ]:
import numpy as np
import numpy.random as rnd

np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

创建一个非线性数据

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)  #degree=2 选择2阶特征
X_poly = poly_features.fit_transform(X)    

对特征进行转换,1阶的x特征转换成 [x,$x^2$]  
然后使用线性回归来进行训练

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_   #返回截距,系数

多项式相比简单的线性回归,还能找出特征之间的关系
e.g., a,b两项参数

In [ ]:
poly_features = PolynomialFeatures(degree=2, include_bias=False)
x=np.array(range(1,11)).reshape((5,2))
x_poly = poly_features.fit_transform(x)

输出的形式是[a,b,$a^2$,ab,$b^2$]

多项式将一个包含n个特征的数列转换成包含:
$ \dfrac{(n+d)!}{d!\,n!} $ 个特征的数列  
其中degree=d

## 学习曲线(Learning Curve)

区分一个模型是过拟合还是欠拟合的办法是:  
如果模型在训练数据上表现较好,但在交叉验证时表现欠佳,则为过拟合  
如果模型在训练数据和交叉验证时都表现欠佳,则为欠拟合

另一种方法是学习曲线: 用不同数量的训练集子集来多次训练模型,并记录模型表现

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)   
    plt.xlabel("Training set size", fontsize=14) 
    plt.ylabel("RMSE", fontsize=14)
    
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)
plt.axis([0, 80, 0, 3])                         
save_fig("underfitting_learning_curves_plot")   
plt.show()    

一条线,测试在m个样本数据上模型训练并测试的效果  
另一条先,测试在m个样本数据上模型的训练效果, 和在验证集(20%)中测试的效果

如果一个模型对于训练数据欠拟合,那么即使添加更多的训练数据也没有用

而一种解决模型过拟合的方法是添加更多的训练数据,直到验证数据误差接近训练误差

### Bias/Variance Tradeoff

一个模型的误差可以表现为三种不同类型的误差:

1.Bias:
这类误差是由错误的假设引起的,例如:当模型为二阶方程时,假设模型是线性模型  
一个高bias的模型很可能会欠拟合

2.Variance:
这类误差是由对训练数据的细小变动过于敏感所引起的,例如:模型拥有过高的自由度(多项式模型中过高的阶)
一个高variance的模型通常会过拟合

Irreducible error:
这类误差是由数据本身的噪音所造成的
减少该类误差的唯一方法只有是对数据进行清洗

## 正则化线性模型 

一个有效应对模型过拟合的办法是对模型进行正则化:  
模型拥有的自由度越少,则模型越不可能过拟合  
例如: 一个简单的多项式模型正则化方法是减小多项式的阶

### 岭回归(Ridge Regression)

岭回归是线性回归的一种正则化版本    
在损失函数中添加了一个正则化项,该项为:  $ \alpha \sum_{i=1}^{n}{\theta_i^2}$  
该正则化迫使模型在考虑拟合数据的同时,还必须使权重尽可能的小  
正则化仅在训练模型的过程中添加进损失函数,当评估模型的使用效果时,正则化项不在考虑范畴中

岭回归损失函数:  
$
J(\mathbf{\theta}) = \text{MSE}(\mathbf{\theta}) + \alpha \dfrac{1}{2}\sum\limits_{i=1}^{n}\theta_i^2
$  
其中,$\alpha$控制了模型的正则化程度
$\alpha=0$,则变为线性模型 , $\alpha$过大,则所有的权重都趋近于0   
$\theta_0$不需要正则化(正则化起始于i=1)

如果定义$\mathbf{w}$为特征权重 ($\theta_1$到$\theta_n$) 的向量  
则正则化项简单等于  
$\large\frac{1}{2}(|| \mathbf{w} ||_2)^2$ , 其中$\large||·||_2$表示权重向量的$l_2$ norm 形式  
对于梯度下降法,只需要在MSE梯度向量后添加 $\alpha\mathbf{w}$ 即可

注:值得注意的是,岭回归对输入特征数据的大小十分敏感,在使用岭回归之前,需要对数据进行缩放(使用StandardScaler)  
几乎所有正则化模型都要求对输入数据进行缩放!

#### 对正规公式法添加岭回归正则化

$
\hat{\mathbf{\theta}} = (\mathbf{X}^T \cdot \mathbf{X} + \alpha \mathbf{A})^{-1} \cdot \mathbf{X}^T \cdot \mathbf{y}
$  
其中$\mathbf{A}$为一个除了左上角第一格为0,其余对角线为1的n×n单位矩阵

使用公式法进行岭回归正则化

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

使用随机梯度下降法进行岭回归正则化

In [ ]:
sgd_reg = SGDRegressor(penalty="l2", random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

penalty="l2" 表示添加一个正则化的项到损失函数,该项等于$l_2$ norm的平方乘以$\frac{1}{2}$ , 其结果正为岭回归  
其中,    
$l_2$ norm:   $||\mathbf{w}||_2 = (w_0^2+w_1^2+···+w_n^2)^{1/2}$ ,也记为 $||·||_2$  
$l_k$ norm:   $||\mathbf{w}||_k = (|w_0|^k+|w_1|^k+···+|w_n|^k)^{1/k}$  
k的值越大,则对outlier(离群点)越敏感